In [1]:
import datasets


asd


In [2]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

# Print all the available datasets
# print(list_datasets())

# # Load a dataset and print the first examples in the training set
# squad_dataset = load_dataset('squad')
# print(squad_dataset['train'][0])
#
# # List all the available metrics
# print(list_metrics())
#
# # Load a metric
# squad_metric = load_metric('squad')

In [3]:
bc_dataset = load_dataset("bookcorpus")



Reusing dataset bookcorpus (/Users/peter/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/af844be26c089fb64810e9f2cd841954fd8bd596d6ddd26326e4c70e2b8c96fc)


In [4]:
print(bc_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 74004228
    })
})


In [5]:
import io
import torch
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


In [6]:
tokenizer = get_tokenizer('basic_english')

In [7]:
sample_texts = bc_dataset['train']['text'][:100]

In [8]:
vocab = build_vocab_from_iterator(map(tokenizer,iter(sample_texts)))

100lines [00:00, 39938.15lines/s]


In [9]:
def data_process(raw_text_iter):
    data = [torch.tensor([vocab[token] for token in tokenizer(item)], dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


train_data = data_process(iter(sample_texts))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
train_data = batchify(train_data, batch_size)


In [11]:
from main import Transformer_Decoder


In [12]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [13]:
ntokens = len(vocab.stoi) # the size of vocabulary
print(ntokens)
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
model = Transformer_Decoder(ntokens, emsize, nhead, nhid, nlayers).to(device)

496


In [22]:

import torch.nn as nn
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        print(batch, i)
        data, targets = get_batch(train_data, i)
        print(data.shape)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        # log_interval = 1
        # if batch % log_interval == 0 and batch > 0:
        cur_loss = total_loss / (batch+1)
        elapsed = time.time() - start_time
        print('| epoch {:3d} | {:5d}/{:5d} batches | '
              'lr {:02.2f} | ms/batch {:5.2f} | '
              'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                elapsed * 1000 / (batch+1),
                cur_loss, math.exp(cur_loss)))
        total_loss = 0
        start_time = time.time()

In [23]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None
import math
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    print('-' * 89)
    scheduler.step()

0 0
torch.Size([35, 20])
| epoch   1 |     0/    1 batches | lr 5.00 | ms/batch 98.50 | loss  5.45 | ppl   233.75
1 35
torch.Size([28, 20])
| epoch   1 |     1/    1 batches | lr 5.00 | ms/batch 33.70 | loss  2.53 | ppl    12.50
-----------------------------------------------------------------------------------------
0 0
torch.Size([35, 20])
| epoch   2 |     0/    1 batches | lr 4.51 | ms/batch 97.00 | loss  4.82 | ppl   123.77
1 35
torch.Size([28, 20])
| epoch   2 |     1/    1 batches | lr 4.51 | ms/batch 34.37 | loss  2.71 | ppl    15.03
-----------------------------------------------------------------------------------------
0 0
torch.Size([35, 20])
| epoch   3 |     0/    1 batches | lr 4.29 | ms/batch 83.31 | loss  4.83 | ppl   125.38
1 35
torch.Size([28, 20])
| epoch   3 |     1/    1 batches | lr 4.29 | ms/batch 34.02 | loss  2.69 | ppl    14.71
-----------------------------------------------------------------------------------------


/usr/local/Cellar/python@3.8/3.8.6_1/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
